In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
##PART 1

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
df = pd.read_excel("/kaggle/input/commute.xlsx")

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
##WE see no missing values in the dataset

In [ ]:
df.dtypes

In [ ]:
##Latitutes and Longitudes are not exactly numbers, but instead they will be treated as co-ordinates, so we will not
#evaluate them as numbers as such

In [ ]:
df.shape

In [ ]:
df['trip_id'].nunique()

In [ ]:
df['customer_id'].nunique()

In [ ]:
#19139 customers

In [ ]:
df.groupby('customer_id').agg({'trip_id':'count'}).sort_values(by='trip_id',ascending=False).head(50).plot.bar(figsize=(20,10))

In [ ]:
##Above we see top 50 customers based on the number of rides they have taken

In [ ]:
##We see that Customer 279 has actually taken a lot more rides than all the other customers

In [ ]:
##similarly we can find the customers which have taken the least number of rides:

df.groupby('customer_id').agg({'trip_id':'count'}).sort_values(by='trip_id',ascending=True).head(50).plot.bar(figsize=(20,10))

In [ ]:
##so, we see that the least number of rides is actually only 1, which means that there are quite a few customers who just
#took a single ride, so let's find out how many customers took only one ride until now

In [ ]:
dfx = df.groupby('customer_id').agg({'trip_id':'count'}).reset_index()
(dfx[dfx['trip_id']==1].shape[0]/dfx.shape[0])*100

In [ ]:
##so, 52.3% (approx) of customers took only one ride after installing the app, until now
##This is a business insight to note and this will help us target these customers for offers

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.groupby('customer_id').agg({'travel_distance':'mean'}).sort_values(by='travel_distance',ascending=False).head(100).plot.bar(figsize=(20,10))

In [ ]:
##Top 100 customers by the mean of their travel distance, showcased above

In [ ]:
df.groupby('customer_id').agg({'travel_distance':'mean'}).sort_values(by='travel_distance',ascending=True).head(100).plot.bar(figsize=(20,10))

In [ ]:
#Bottom 100 customers by the mean of their travel distance, showcased above

In [ ]:
##So, we notice that one customer has traveled a mean distance of -1. Now, this could mean two things: Either the data was entered
#incorrectly, or else the customer travelled in the wrong direction(opposite to the one shown in the map) for 1 KM
##It could actually mean other things as well, but at this moment, we can understand that something did go wrong with this
#customer
##And the other customers who have not traveled at all, have traveled distance as 0. But, again, these customers do have
#co-ordinates in the data, which means that they have surely booked rides, and then cancelled them
#so, again an insight to note, that quite a few customers have cancelled rides, which is not a good sign.
##These users again become a target, for promoting offers, so that they start travelling

In [ ]:
df.groupby('customer_id').agg({'travel_time':'mean'}).sort_values(by='travel_time',ascending=False).head(100).plot.bar(figsize=(20,10))

In [ ]:
df[['travel_distance','travel_time','trip_fare']].corr()  ##Checking correlation among numeric variables

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df[['travel_distance','travel_time','trip_fare']].corr(),annot=True)
plt.show()

In [ ]:
##We see that quite obviously, travel distance and trip fare are very much correlated to each other

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df[['pick_lat','pick_lng','drop_lat','drop_lng']].corr(),annot=True)
plt.show()

In [ ]:
##We do see correlation among latitudes and longitudes, which is again very obvious because latitudes and longitudes are
#used to measure the position of the vehicle/person

In [ ]:
df.head()

In [ ]:
dfcopy = df

In [ ]:
import time
import datetime
from datetime import timedelta
dfcopy['timestamp'] = dfcopy['timestamp'].apply(lambda d: time.strftime('%m/%d/%Y %H:%M:%S',  time.gmtime(d/1000.)))

In [ ]:
dfcopy.head()

In [ ]:
sns.pairplot(data=dfcopy)
plt.show()

In [ ]:
##So, if we see the last row, we can understand that the trip fare does not depend on the pick location, but trip fare
#does depend on travel distance, but not travel time (which is a good thing for customers in busy citis with lots of traffic
#since they won't have to pay more for spending time in the traffic)
##Travel distance does not depend on pick and drop locations in any way, since we do see a properly spread cluster of values in
#the scatterplot, and this is quite obvious in a real life perspective
#Latitude VS Longitude scatterplot is not much interpretable at this moment(since it would be better to check it out
#on a map, but of course the spread of values in the plot is what shows that trips are being taken all across the city/country
#and not just in some specific location--of course, there might be ceratin areas where trips might be more, and we'll check
#that out using clustering)

In [ ]:
dfcopy.dtypes[dfcopy.dtypes!=object].index

In [ ]:
for i in dfcopy.dtypes[dfcopy.dtypes!=object].index:
    sns.boxplot(df[i])
    plt.show()

In [ ]:
##Latitudes and longitudes are definitely bound to have outliers, because there would be people taking trips from remote
#areas of the city, almost everyday, but the number of those people would be very less, which is acceptable, and hence these
#outliers are still important for our predictions then
##Travel distance also sees a lot of outiers, particularly above the upper whisker, which could definitely signify that
#certain people do prefer to take trips only when they have to travel to a very far off place, or let's say if they are late,
#and they feel like they won't be able to make it in time to the far off place--this could be one of the many insights

In [ ]:
dfcopy['timestamp'].apply(lambda d: str(d)[6:10]).unique()

In [ ]:
##So, we see we have data only for 2019 at the moment. If we had had data for other years as well, I would have created
#a column for year too, for analyzing it through the years
##For now, let's check the months:
dfcopy['month'] = dfcopy['timestamp'].apply(lambda d: str(d)[3:5])

In [ ]:
dfcopy.head()

In [ ]:
df.groupby(['customer_id','month']).agg({'trip_id':'count'}).sort_values(by='trip_id',ascending=False).head(20)

In [ ]:
##Above we can see that our top customers have travelled in which month, and how many trips they have taken in that month

In [ ]:
df.groupby('month').agg({'trip_id':'count'}).plot()

In [ ]:
##Above we can see the trend analysis of trips across months in the year 2019
##Trips shot up from January to February (which gives us an insight to work upon offers, etc. in January)
##and of course, if people are travelling extensively in February, we can try to cool down on the offers(since people are
#already willing to travel much in this period--based on the trend)
##Trips started falling in April and came to the lowest in June, but then again shot up in July
##Again, it is very much possible that new offers were introduced in these months, which caused the trips to shoot up, or maybe
##like based on the weather conditions in February, people started taking more rides
##July is again a part of monsoon, so heavy rains could be a reason for trips shooting up. There could be many such reasons
#which we can analyze based on which locations the data was collected from and what were the social, economic and weather
#conditions in that area at the period of time which is being analyzed

In [ ]:
df.head()

In [ ]:
dfc  = df.groupby('customer_id').agg({'trip_id':'count'}).reset_index()

In [ ]:
dfc[dfc['trip_id']>=2]['customer_id'].values

In [ ]:
dfimp = df.set_index('customer_id').loc[dfc[dfc['trip_id']>=2]['customer_id'].values].reset_index()

In [ ]:
dfimp.sort_values(by='travel_time',ascending=True)
##To check whether the person travelled or not

In [ ]:
dfimp.sort_values(by='trip_fare',ascending=True)
##To check whether the person travelled or not

In [ ]:
##So, now we know that in dfimp, people have travelled at least twice, and none of these trips were cancelled

In [ ]:
dfimp['customer_id'].nunique()

In [ ]:
dfimp.shape[0]

In [ ]:
dfimp['customer_id'].unique()

In [ ]:
import time
import datetime
from datetime import timedelta
dfimp['timestamp'] = dfimp['timestamp'].apply(lambda d: time.strftime('%m/%d/%Y %H:%M:%S',  time.gmtime(d/1000.)))

In [ ]:
dfimp

In [ ]:
dfimp[dfimp['customer_id']=='CUST_001'].head(2).iloc[0]['timestamp']
##Trying for one customer

In [ ]:
difflist = []
for i in dfimp['customer_id'].unique():
    datetimeFormat = '%m/%d/%Y %H:%M:%S'
    dat1 = dfimp[dfimp['customer_id']==i].sort_values(by='timestamp',ascending=True).head(2).iloc[0]['timestamp']
    dat2 = dfimp[dfimp['customer_id']==i].sort_values(by='timestamp',ascending=True).head(2).iloc[1]['timestamp']
    diff = datetime.datetime.strptime(dat2, datetimeFormat) - datetime.datetime.strptime(dat1, datetimeFormat)
    difflist.append(diff)

In [ ]:
difflist  #(based on whether the first value was greater or the second value)

In [ ]:
##Above I have chosen the first two trips of all the people and calculated the time between those two trips
##Now, we will calculate the average time spent by customers between their first two trips

In [ ]:
np.array(difflist).mean()

In [ ]:
##WE can see that average duration between the 1st trip and the 2nd trip of customers is 
##days=1, seconds=2829, microseconds=693538

In [ ]:
df.head()

In [ ]:
data = df[['trip_fare','travel_distance','travel_time']]

In [ ]:
data.head()

In [ ]:
y = data['trip_fare']
X = data.drop(columns='trip_fare')

In [ ]:
X.head()

In [ ]:
##We'll be building a linear regression model
##Let us build a stats model first, to understand which variables are of importance here
##The accuracy in case of Linear Regression is given by the Rsquare achieved (adjusted Rsquare is a measure obtained after
#dropping the importance of the variables which are overfitting the model--but it does not neccessarily mean that the overfit
#would have been removed)

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import statsmodels.api as sm

model1 = sm.OLS(y,X).fit()
model1.summary()

In [ ]:
model1.predict(X)

In [ ]:
y_train_pred = model1.predict(X)

In [ ]:
X_test = pd.DataFrame(data=[3.5,15]).T.rename(columns={0:'travel_distance',1:'travel_time'})
X_test

In [ ]:
model1.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y,y_train_pred))

In [ ]:
##Above we see the predicted trip_fare for a trip with travel_distance of 3.5 kms and travel_time of 15 minutes

In [ ]:
##So, we see that the Rsquare and the adjusted Rsquare is the same, which basically means that none of the variables were
#causing any overfit, and that the variables were not correlated(which we also saw using a Heatmap in the beginning)

In [ ]:
##Now, let's build a machine learning model so we can predict out value

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X,y)

In [ ]:
X_test = pd.DataFrame(data=[3.5,15]).T.rename(columns={0:'travel_distance',1:'travel_time'})
X_test

In [ ]:
result = lr.fit(X,y)
y_test_pred = lr.predict(X_test)
y_train_pred = lr.predict(X)

In [ ]:
##The trip_fare for a trip with travel_distance of 3.5 kms and travel_time of 15 minutes is shown below:

In [ ]:
y_test_pred

In [ ]:
##Accuracy of the Machine Learning (regression) model is shown below:

In [ ]:
from sklearn.metrics import r2_score
r2_score(y,y_train_pred)

In [ ]:
##Root mean square error for the Model built, is show below:

In [ ]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y,y_train_pred))

In [ ]:
##So, above we saw that the predicted values given by the stats model and the machine learning regression were different
##And that was because the accuracy achieved in both the cases were different
##Ideally, we desire a model that has high accuracy (Rsquare) without overfit, and a lower RMSE
##So, we see that on building the machine learning model, the RMSE does decrease by a little amount, but the Rsquare
#decreases by a large amount
##so, in this particular case, we will select our Statistics Model as the preferred one

In [ ]:
df.head()

In [ ]:
import numpy as np
import pandas as pd
import folium
import webbrowser
import os
import math

from h3 import h3
from folium import Map

In [ ]:
df.head()

In [ ]:
pos = df[['pick_lat','pick_lng','drop_lat','drop_lng']]

In [ ]:
pos.isna().sum()

In [ ]:
##Creating radian latitude and logitude columns now

In [ ]:
pos['lngpick'] = np.radians(pos['pick_lng'].to_numpy())
pos['latpick'] = np.radians(pos['pick_lat'].to_numpy())
pos['lngdrop'] = np.radians(pos['drop_lng'].to_numpy())
pos['latdrop'] = np.radians(pos['drop_lat'].to_numpy())

In [ ]:
pos.head()

In [ ]:
pos1 = pos[['lngpick','latpick']]
pos2 = pos[['lngdrop','latdrop']]

In [ ]:
##Setting parameters for DBSCAN:
eps_in_meters = 50.0
num_samples = 10

In [ ]:
##First finding out where most of the trips started:
from sklearn.cluster import DBSCAN

earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

pos1['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, 
                           metric='haversine').fit_predict(pos1[['lngpick', 'latpick']])

In [ ]:
h3_level = 8 ##as defined in the question

In [ ]:
##Function to convert latitude and longitudes to H3 key:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['lngpick'], row['latpick'], h3_level)

In [ ]:
pos1['h3'] = pos1.drop(columns='cluster').apply(lat_lng_to_h3, axis=1)

In [ ]:
pos1

In [ ]:
##Selecting only the locations that belong to a DBSCAN-generated cluster. Clusters marked with -1 are noise in this case:

In [ ]:
df1 = pos1[pos1.cluster != -1].copy()

In [ ]:
df1['h3'].value_counts()  ##H3 levels for the positions where the pick location was

In [ ]:
df1['cluster'].value_counts().head(5) ##Top 5 clusters for the positions where the pick location was

In [ ]:
##Now, for the Drop location where most trips ended:



In [ ]:
pos.head()

In [ ]:
pos1 = pos[['lngpick','latpick']]
pos2 = pos[['lngdrop','latdrop']]

In [ ]:
##Setting parameters for DBSCAN:
eps_in_meters = 50.0
num_samples = 10

In [ ]:
##First finding out where most of the trips started:
from sklearn.cluster import DBSCAN

earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

pos2['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, 
                           metric='haversine').fit_predict(pos2[['lngdrop', 'latdrop']])

In [ ]:
h3_level = 8 ##as defined in the question

In [ ]:
##Function to convert latitude and longitudes to H3 key:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['lngdrop'], row['latdrop'], h3_level)

In [ ]:
pos2['h3'] = pos2.drop(columns='cluster').apply(lat_lng_to_h3, axis=1)

In [ ]:
pos2

In [ ]:
##Selecting only the locations that belong to a DBSCAN-generated cluster. Clusters marked with -1 are noise in this case:

In [ ]:
df2 = pos2[pos2.cluster != -1].copy()

In [ ]:
df2['h3'].value_counts()  ##H3 levels for the positions where the pick location was

In [ ]:
df2['cluster'].value_counts().head(5) ##Top 5 clusters for the positions where the pick location was

In [ ]:
##So, we have calculated top 5 clusters based on pick locations and drop locations
##We do see that people are usually travelling from one specific location to another, and this is actually true, since
#quite a lot of people will be taking trips from highly occupied residential areas, to their workplace/institutes, etc.
##We can take an example of people travelling from HSR layout/Koramangala to various co-working spaces in Bangalore, 
#so the amount of #picks from residential areas could be highest, and the amount of drops at the co-working spaces could be
#highest